# SIGNATE Competition Workflow Template

このノートブックは、Google Colab上でGitHubリポジトリを同期させながらSIGNATEコンペティションに取り組むためのテンプレートです。

## 事前準備
1. **Colab Secrets (鍵アイコン)** に以下の値を設定してください：
    * `GITHUB_TOKEN`: GitHub Personal Access Token (repoスコープ付き)
    * `USER_EMAIL`: Gitコミット用のメールアドレス
    * `USER_NAME`: Gitコミット用のユーザー名
    * `REPO_URL`: リポジトリのHTTPS URL (例: `https://github.com/username/repo.git`)
2. **Google Drive** に `signate.json` を保存してください (例: `MyDrive/Signate/signate.json`)。

## 1. 環境設定とリポジトリのクローン

In [ ]:
from google.colab import userdata
import os

# Colab Secretsから認証情報を取得
try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    USER_EMAIL = userdata.get('USER_EMAIL')
    USER_NAME = userdata.get('USER_NAME')
    REPO_URL = userdata.get('REPO_URL')
except Exception as e:
    print("Secretsが設定されていません。左側の鍵アイコンから設定してください。")
    raise e

# リポジトリ名を取得
REPO_NAME = REPO_URL.split('/')[-1].replace('.git', '')

# Gitのユーザー設定
!git config --global user.email "$USER_EMAIL"
!git config --global user.name "$USER_NAME"

# リポジトリのクローン (またはPull)
if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_NAME}...")
    # URLにトークンを埋め込む
    clone_url = REPO_URL.replace("https://", f"https://{USER_NAME}:{GITHUB_TOKEN}@")
    !git clone {clone_url}
    %cd {REPO_NAME}
else:
    print(f"{REPO_NAME} already exists. Pulling latest changes...")
    %cd {REPO_NAME}
    !git pull origin main

## 2. 依存ライブラリのインストール

In [ ]:
!pip install signate
# !pip install -r requirements.txt # 必要に応じてコメントアウトを外す

## 3. SIGNATE APIの設定 (Google Drive経由)

In [ ]:
from google.colab import drive
import shutil

# Google Driveをマウント
drive.mount('/content/drive')

# ディレクトリ作成
os.makedirs('/root/.signate', exist_ok=True)

# signate.jsonのパス (自身の環境に合わせて変更してください)
SIGNATE_JSON_PATH = '/content/drive/MyDrive/Signate/signate.json'

if os.path.exists(SIGNATE_JSON_PATH):
    shutil.copy(SIGNATE_JSON_PATH, '/root/.signate/signate.json')
    os.chmod('/root/.signate/signate.json', 0o600)
    print("SIGNATE API setup complete.")
else:
    print(f"Error: {SIGNATE_JSON_PATH} not found. Please check the path.")

## 4. データダウンロード

In [ ]:
# コンペティションキー (固定)
COMPETITION_KEY = '092375ab3c4a43c18c8277e1fd264aa9'

# タスクリストの確認
!signate task-list --competition_key={COMPETITION_KEY}

In [ ]:
# ファイルのダウンロード (タスクキーとファイルキーを指定)
# !signate download --task_key=<TASK_KEY> --file_key=<FILE_KEY> --path=./data/

## 5. 作業スペース (Analysis & Modeling)
ここでコードを記述・実行します。

In [ ]:
# Your code here
import pandas as pd
import numpy as np

print("Hello, SIGNATE!")

## 6. 結果の提出 (Submission)

In [ ]:
# 提出コマンドの実行
# !signate submit --task_key=<TASK_KEY> ./submission.csv --memo "Colab submission"

## 7. GitHubへの同期 (Push)
作業内容をGitHubに保存します。

In [ ]:
!git status

In [ ]:
# 変更をステージング
!git add .

# コミットメッセージを入力してコミット
commit_message = "Update from Colab"
!git commit -m "{commit_message}"

# プッシュ
!git push origin main